In [1]:
import numpy as np
import pandas as pd
import random

In [2]:
# Define boiler parameters
coal_input = np.random.randint(50, 150, 500)  # kg/hr
primary_air_flow = np.random.randint(200, 500, 500)  # m3/hr
secondary_air_flow = np.random.randint(100, 300, 500)  # m3/hr
spray_water_flow = np.random.randint(10, 50, 500)  # kg/hr
flue_gas_temp = np.random.randint(120, 250, 500)  # Celsius
excess_oxygen = np.random.uniform(2.0, 6.0, 500)  # %

In [3]:
# Compute efficiency using a synthetic function
def compute_efficiency(coal, air1, air2, spray, flue_temp, oxy):
    base_eff = 80  # Base efficiency in %
    efficiency = base_eff + (50 / coal) - (flue_temp / 100) - (spray / 20) + (oxy / 10)
    return round(min(100, max(50, efficiency)), 2)  # Clamp between 50 and 100%

In [4]:
boiler_efficiency = [compute_efficiency(coal_input[i], primary_air_flow[i], secondary_air_flow[i], 
                                        spray_water_flow[i], flue_gas_temp[i], excess_oxygen[i])
                     for i in range(500)]


In [5]:
boiler_efficiency

[76.44,
 77.33,
 78.58,
 77.31,
 77.88,
 77.4,
 77.37,
 78.06,
 78.6,
 77.04,
 79.3,
 77.81,
 77.82,
 77.15,
 78.12,
 76.69,
 77.77,
 77.4,
 78.74,
 76.96,
 76.71,
 78.77,
 77.46,
 77.2,
 76.96,
 77.95,
 76.82,
 78.02,
 77.28,
 78.92,
 76.98,
 77.8,
 77.61,
 76.93,
 78.39,
 77.43,
 78.48,
 78.58,
 76.72,
 77.91,
 77.99,
 78.07,
 78.35,
 76.98,
 77.98,
 78.49,
 77.22,
 77.47,
 77.7,
 76.01,
 77.09,
 77.56,
 77.93,
 76.7,
 78.76,
 77.19,
 76.12,
 77.43,
 78.27,
 78.44,
 77.86,
 76.47,
 76.92,
 78.65,
 77.48,
 78.24,
 76.81,
 78.0,
 77.97,
 76.69,
 77.75,
 78.5,
 77.12,
 77.13,
 76.86,
 76.48,
 78.67,
 78.69,
 78.12,
 77.79,
 77.84,
 78.1,
 77.61,
 77.47,
 77.22,
 78.25,
 77.8,
 77.38,
 77.96,
 77.71,
 76.31,
 79.29,
 77.74,
 77.74,
 77.19,
 77.35,
 77.16,
 76.88,
 78.66,
 78.31,
 78.5,
 77.84,
 78.18,
 76.54,
 77.87,
 77.26,
 78.03,
 76.33,
 77.54,
 77.14,
 76.69,
 76.25,
 78.43,
 76.39,
 76.62,
 78.93,
 77.8,
 79.17,
 78.99,
 77.35,
 77.14,
 77.95,
 77.36,
 78.31,
 78.18,
 77.68,
 77.5,

In [6]:
# Create DataFrame
data = pd.DataFrame({
    'Coal_Input': coal_input,
    'Primary_Air_Flow': primary_air_flow,
    'Secondary_Air_Flow': secondary_air_flow,
    'Spray_Water_Flow': spray_water_flow,
    'Flue_Gas_Temp': flue_gas_temp,
    'Excess_Oxygen': excess_oxygen,
    'Efficiency': boiler_efficiency
})

In [7]:
# Define state space (Efficiency levels) and action space (Parameter adjustments)
efficiency_levels = np.arange(50, 101, 5)
actions = ['Increase_Coal', 'Decrease_Coal', 'Increase_Air', 'Decrease_Air', 'Increase_Spray', 'Decrease_Spray']

In [8]:
# Initialize Q-table
q_table = {eff: {act: 0 for act in actions} for eff in efficiency_levels}

In [9]:
# Q-learning parameters
learning_rate = 0.1
discount_factor = 0.9
epsilon = 0.2  # Exploration rate

In [11]:
# Training loop
for _ in range(1000):  # 1000 episodes
    state = random.choice(efficiency_levels)
    for _ in range(50):  # 50 steps per episode
        action = random.choice(actions) if random.uniform(0, 1) < epsilon else max(q_table[state], key=q_table[state].get)
        
        # Simulate efficiency change based on action
        if 'Increase' in action:
            next_state = min(100, state + random.randint(1, 3))
        else:
            next_state = max(50, state - random.randint(1, 3))
        
        # Ensure next_state is a valid efficiency level
        valid_efficiency_levels = list(q_table.keys())
        next_state = min(valid_efficiency_levels, key=lambda x: abs(x - next_state))
        
        reward = next_state - state  # Reward = improvement in efficiency
        q_table[state][action] += learning_rate * (reward + discount_factor * max(q_table[next_state].values()) - q_table[state][action])
        state = next_state

In [12]:
# Function to get best settings for a given efficiency target
def suggest_optimal_settings(target_efficiency):
    if target_efficiency not in q_table:
        print("Efficiency target out of range. Choose between 50 and 100.")
        return
    best_action = max(q_table[target_efficiency], key=q_table[target_efficiency].get)
    print(f"To achieve {target_efficiency}% efficiency, the best action is: {best_action}")

In [27]:
# Example user input
suggest_optimal_settings(65)

To achieve 65% efficiency, the best action is: Increase_Spray
